In [1]:
# 2/29/24 Transverse-Field Ising Model (TFIM) 16 qubits
# calling qiskit packages, make sure to use qiskit v0.46.0 as that is the only version that supports execute_function
from qiskit import QuantumCircuit, Aer, execute
import json

# setting cirucit name to avoid having to change
name = "tfim"

# reading qasm file
qc = QuantumCircuit.from_qasm_file(name + "_16.qasm")

# using statevector simulator
backend = Aer.get_backend("statevector_simulator")

# setting shot increments
x = 64

# simple while loop to calculate 
while x <= 1024:
    # actually running the circuit with the AER statevector and with the specified shot
    job = execute(qc, backend, shots=x)
    result = job.result()

    # statevector for debugging 
    statevector = result.get_statevector()
    print(statevector)

    # getting counts
    counts = result.get_counts(qc)

    # exporting counts to json file
    with open(name + '_counts_' + str(x) + '_shots.json', 'w', encoding='utf-8') as f:
        json.dump(counts, f, ensure_ascii=False, indent=4)
    
    x += 64

/tmp/ipykernel_55688/3808466171.py:13: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  backend = Aer.get_backend("statevector_simulator")
/tmp/ipykernel_55688/3808466171.py:21: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(qc, backend, shots=x)


Statevector([-7.65466754e-01-6.43067028e-01j,
             -1.80207140e-16+4.76943551e-16j,
             -2.82610637e-15-2.38452074e-15j, ...,
             -1.68913850e-19-3.47132313e-19j,
              1.96770655e-18-4.89402105e-19j,
             -7.98819576e-19+1.47246722e-19j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))
Statevector([-7.65466754e-01-6.43067028e-01j,
             -1.80207140e-16+4.76943551e-16j,
             -2.82610637e-15-2.38452074e-15j, ...,
             -1.68913850e-19-3.47132313e-19j,
              1.96770655e-18-4.89402105e-19j,
             -7.98819576e-19+1.47246722e-19j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))
Statevector([-7.65466754e-01-6.43067028e-01j,
             -1.80207140e-16+4.76943551e-16j,
             -2.82610637e-15-2.38452074e-15j, ...,
             -1.68913850e-19-3.47132313e-19j,
              1.96770655e-18-4.89402105e-19j,
             -7.98819576e-19+1.47246722e-19j],
            dims=(2,